In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import sklearn
from sklearn.linear_model import LogisticRegression
import numpy as np

In [ ]:
!gdown "https://drive.google.com/uc?id=1a0e2Tp6Yiv3KXKHtfe4JlgCRW3UWoBpU"
!unzip /content/European-soccer-data.zip

#Data Gathering

First we need to extract our dataset, since it's in the form of a sqlite databse
we need to establish a connection to the database first and from there manipulate our connection to
extract the data through queries

In [ ]:
connection  = sqlite3.connect("database.sqlite")

cursor = connection.cursor()

query = "SELECT * FROM Player;"
cursor.execute(query)
players = pd.read_sql_query(query, connection)

query = "SELECT * FROM Player_Attributes;"
cursor.execute(query)
player_attr = pd.read_sql_query(query, connection)

query = "SELECT * FROM Match;"
cursor.execute(query)
matches = pd.read_sql_query(query, connection)

query = "SELECT * FROM League;"
cursor.execute(query)
leagues = pd.read_sql_query(query, connection)

query = "SELECT * FROM Country;"
cursor.execute(query)
countries = pd.read_sql_query(query, connection)

query = "SELECT * FROM Team;"
cursor.execute(query)
teams = pd.read_sql_query(query, connection)

query = "SELECT * FROM Team_Attributes;"
cursor.execute(query)
team_attr = pd.read_sql_query(query, connection)

#Data cleaning

Getting rid of any row that has an NaN value in it does help us out since
we can move forward with the assumption of clean data, but it will incur
some data loss but since we only care about the appearnce of NaN values in specific columns, that can reduce the data loss

Lets first clean up the data by parsing out only the information that we really need, and leaving out the data
that won't have much of an impact

In [ ]:
print(len(matches.columns))
matches.drop(matches.columns[77:115], axis=1, inplace= True)
matches.drop(matches.columns[11:55], axis=1, inplace= True)
print(len(matches.columns))


Now we can drop rows that contain NaN values since we'll now know that it's
only for data we care about

In [ ]:
print(len(matches))
matches = matches.dropna()
print(len(matches))


#Data Processing

Now we'll maintain a dictionary of players and teams to their stats over time as well as matches to their stats

In [ ]:
players_dict = {}
team_dict =  {}
match_dict = {}
league_dict = {}
country_dict = {}

def getStats(ID,stat_source, data_source):
    data= {}
    stats = stat_source.loc[stat_source[data_source] == ID]
    for date_data in stats.iterrows():
        if data_source == 'match_api_id':
            data = {}
            for index,value in date_data[1].items():
                if index != 'match_api_id':
                    data[index] = value
                if index == 'date':
                    data[index] = value[0:4]
        else:
            year = date_data[1]['date'][0:4]
            if year not in data:
                data[year] = {}
                for index,value in date_data[1].items():
                    data[year][index] = value
    return data

In [ ]:
for row in players.iterrows():
    players_dict[row[1][1]] ={}
    players_dict[row[1][1]]['name'] = row[1]['player_name']
    players_dict[row[1][1]]['birthday'] = row[1]['birthday']
    players_dict[row[1][1]]['height'] = row[1]['height']
    players_dict[row[1][1]]['weight'] = row[1]['weight']
    players_dict[row[1][1]]['overall_stats'] = getStats(row[1][1],player_attr,players.columns[1])

for row in teams.iterrows():
    team_dict[row[1][1]] ={}
    team_dict[row[1][1]]['full_name'] = row[1]['team_long_name']
    team_dict[row[1][1]]['short_name'] = row[1]['team_short_name']
    team_dict[row[1][1]]['overall_stats'] = getStats(row[1][1],team_attr,team_attr.columns[2] )

for row in matches.iterrows():
    match_dict[row[1][6]] ={}
    match_dict[row[1][6]] = getStats(row[1][6],matches,matches.columns[6])

#Feel free to try and convert the rest of the tables into this format!


#Basic data analysis

Now that we've oragnized the data into an eaiser form to work with from, we can being to answer some questions about the data wuch as win percentages of teams

In [ ]:
wins_dict = {}

for key in team_dict:
    wins_dict[key] = {}
    wins_dict[key]['wins'] = 0
    wins_dict[key]['games'] = 0
    wins_dict[key]['percentage'] = 0
    wins_dict[key]['name'] = team_dict[key]['full_name']

for key in match_dict:
    #find the two teams
    home_team_api = match_dict[key]['home_team_api_id']
    away_team_api =match_dict[key]['away_team_api_id']
    wins_dict[home_team_api]['games'] +=1
    wins_dict[away_team_api]['games'] +=1
    if match_dict[key]['home_team_goal'] >  match_dict[key]['away_team_goal']:
        wins_dict[home_team_api]['wins'] +=1
        wins_dict[home_team_api]['percentage'] = wins_dict[home_team_api]['wins']/wins_dict[home_team_api]['games']
    if match_dict[key]['home_team_goal'] <  match_dict[key]['away_team_goal']:
        wins_dict[away_team_api]['wins'] +=1
        wins_dict[away_team_api]['percentage'] = wins_dict[away_team_api]['wins']/ wins_dict[away_team_api]['games']


max_per = 0
best_team = ''

for key in wins_dict:
    if wins_dict[key]['percentage'] > max_per:
        max_per = wins_dict[key]['percentage']
        best_team = wins_dict[key]['name']
print("the team with the best win percentage is",best_team,"with a win percentage of", round(max_per,2))

#TODO: find the team with the lowest win percentage


Here are some more aggregated stats that tell us a bit more about the data

In [ ]:
def get_team_playstyle(team):

    #retrieve some relevant team statistics
    team_features =np.array(['defencePressure', 'buildUpPlaySpeed', 'chanceCreationShooting'])
    total_defencePressure = 0
    buildUpPlaySpeed = 0
    chanceCreationShooting = 0
    num_years = len(team_dict[team]['overall_stats'].keys())
    print(team_dict[team]['overall_stats'] != {})
    if team_dict[team]['overall_stats'] != {}:
        for year in team_dict[team]['overall_stats'].keys():
            total_defencePressure += team_dict[team]['overall_stats'][year][team_features[0]]
            buildUpPlaySpeed += team_dict[team]['overall_stats'][year][team_features[1]]
            chanceCreationShooting += team_dict[team]['overall_stats'][year][team_features[2]]

        average_total_defencePressure =total_defencePressure/num_years
        average_buildUpPlaySpeed =buildUpPlaySpeed/num_years
        average_chanceCreationShooting =chanceCreationShooting/num_years

        team_playstyle = np.array([average_total_defencePressure,average_buildUpPlaySpeed,average_chanceCreationShooting])
    else:
      team_playstyle = np.array([0,0,0])


def get_team_player_averages(team):
    return 42


    #TODO: Find average player rating for team for a given year


#Loading in Data


In [ ]:
X_train = []
y_train = []


print("Starting training data cureation!")
X_train = []  # List of feature vectors
y_train = []  # List of labels

for key in match_dict.keys():
    home_team_id = match_dict[key]['home_team_api_id']
    away_team_id = match_dict[key]['away_team_api_id']
    home_goals = match_dict[key]['home_team_goal']
    away_goals = match_dict[key]['away_team_goal']

    if home_goals > away_goals:
        winning_team = home_team_id
        losing_team = away_team_id
    if home_goals < away_goals:
        losing_team = home_team_id
        winning_team = away_team_id
    losing_team_win_percentage = wins_dict[losing_team]['percentage']
    winning_team_win_percentage = wins_dict[winning_team]['percentage']

    # Randomize the winner order
    if np.random.uniform(0,1) < 0.5:
        X_train.append(np.array([losing_team, winning_team, losing_team_win_percentage,winning_team_win_percentage]))
        y_train.append(0)  # 0 for losing, 1 for winning

    else:
        X_train.append(np.array([winning_team, losing_team, winning_team_win_percentage,losing_team_win_percentage]))
        y_train.append(1)  # 0 for losing, 1 for winning


#Training

In [ ]:
print("Starting Training!")
# Train the logistic regression model
model = sklearn.linear_model.LogisticRegression()
out = model.fit(X_train, y_train)
print("Done!")

#Prediction

In [ ]:
result = ""
X_predict = np.array([8634, 8633, wins_dict[8634]['percentage'],wins_dict[8633]['percentage']]).reshape(1, -1)

out = model.predict(X_predict)
probabilities = model.predict_proba(X_predict)
probability_of_positive_class = probabilities[:, 1]

if out[0] == 0:
    result = "No"
else:
    result = "Yes"

print("Will Team1 beat Team2? ",result," With what? probability? ", round(probability_of_positive_class[0],2))


In [ ]:
#TODO: Now that we have a predictive model for which teams could beat other teams, try and make a simple 8 team knockout style tournament!

teams = [8634, 8633, 8655, 8586, 8658, 10003, 8455, 8483]


In [ ]:
for team1 in teams:
    for team2 in teams:
        print(team1, "vs", team2)
        result = ""
        X_predict = np.array([team1, team2, wins_dict[team1]['percentage'],wins_dict[team2]['percentage']]).reshape(1, -1)

        out = model.predict(X_predict)
        probabilities = model.predict_proba(X_predict)
        probability_of_positive_class = probabilities[:, 1]

        if out[0] == 0:
            result = "No"
        else:
            result = "Yes"

        print("Will Team1 beat Team2? ",result," With what? probability? ", probability_of_positive_class[0])


# Sandbox

Feel free to try something new, maybe add some new data analytics, or try to retrain the model with new combinations of features!